In [4]:
from core.utils import Tibanna
from core import ff_utils
from core.utils import run_workflow
from datetime import datetime
from core.wfr import *

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
tibanna.ff_keys['default']['server'] = 'https://data.4dnucleome.org'
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
tibanna.ff_keys['default']['server'] = 'https://data.4dnucleome.org'
    

In [3]:
import time
from tasks import run_fastqc
from invoke import run

# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

add_wfr = True

# url for hic exps
fastq_url = "search/?type=FileFastq&quality_metric.uuid=No+value" + \
            "&status=released&status=released%20to%20project&status=uploaded" + \
            "&limit=all" + \
            "&q=date_created%3A%3E%3D2016-09-01"

fastq_files = ff_utils.get_metadata(fastq_url , connection=ff)['@graph']



counter = 0
completed = 0
print len(fastq_files)

okay = 0
non = 0
running = 0
for a_fastq in fastq_files:
    report = get_wfr_out(a_fastq['accession'], 'fastqc-0-11-4-1/1', None, ff)
    if report.startswith('no') or not report:
        non += 1
        if add_wfr:
            print 'fastqc running for', a_fastq['accession']
            code_qc= "invoke run_fastqc " + env + " " + a_fastq['display_title'] + " " + a_fastq['uuid']
            run(code_qc)
        else:
            print 'fastqc run missing for', a_fastq['accession']   
    elif report == 'running':
        running += 1
        print(a_fastq['accession'], 'still running')
    else:
        okay += 1
        
print("okay, not okay, running")    
print(okay, non, running)


13
fastqc running for 4DNFIZX777TN
about to start run fastqc-0-11-4-1_4DNFIZX777TN.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 5, 14, 14, 15, 11, 776000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': 'bded6468-57a2-11e8-b534-ab21897cab1e', 'HTTPHeaders': {'x-amzn-requestid': 'bded6468-57a2-11e8-b534-ab21897cab1e', 'content-length': '146', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIZX777TN.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFIZX777TN.fastq.gz
fastqc running for 4DNFIEGUZAHT
about to start run fastqc-0-11-4-1_4DNFIEGUZAHT.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 5, 14, 14, 15, 13, 588000, tzinfo=tzlocal()), 'ResponseMetad

fastqc running for 4DNFI43RP96B
about to start run fastqc-0-11-4-1_4DNFI43RP96B.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 5, 14, 14, 15, 31, 603000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': 'c9bc287d-57a2-11e8-9416-e145cf2585cb', 'HTTPHeaders': {'x-amzn-requestid': 'c9bc287d-57a2-11e8-9416-e145cf2585cb', 'content-length': '146', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFI43RP96B.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:fastqc-0-11-4-1_4DNFI43RP96B.fastq.gz
fastqc running for 4DNFI8KTY3QD
about to start run fastqc-0-11-4-1_4DNFI8KTY3QD.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2018, 5, 14, 14, 15, 33, 554000, tzinfo=tzlocal()), 'ResponseMetadata

In [5]:
# Release fastq qc

fastq_url = '/search/?status=released&status=released%20to%20project&status=archived&type=FileFastq'
fastq_files = ff_utils.get_metadata(fastq_url , connection=ff)['@graph']
fastq_files = [i for i in fastq_files if i.get('quality_metric')]
print len(fastq_files)
success = 0
patch_problem = 0

patch_trigger = True

for fastq in fastq_files:
    fastq_status = fastq['status']
    if fastq_status in ['released', 'released to project', 'archived']:
        qc_status = ff_utils.get_metadata(fastq['quality_metric'],connection=ff)['status']
        if qc_status != fastq_status:
            print('unmatched status', fastq['accession'])
            patch_data = {"status": fastq_status}
            
            if patch_trigger:
                try:
                    ff_utils.patch_metadata(patch_data, obj_id=fastq['quality_metric'] ,connection=ff)
                    success += 1
                    print success
                except Exception as e:
                    print e
                    print
                    patch_problem += 1
print success
print patch_problem



2947
('unmatched status', u'4DNFIGZKCSBB')
1
('unmatched status', u'4DNFIYIREE8L')
2
('unmatched status', u'4DNFI6PJCX1Z')
3
('unmatched status', u'4DNFIBS2562B')
4


4
0
